In [18]:
!pip install tensorflow

In [19]:
!pip install PIL

ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


In [20]:
!pip install scipy

In [21]:
import tensorflow as tf
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [22]:
# Parameter
IMG_SIZE = (224, 224)  # Ukuran gambar untuk MobileNetV2
BATCH_SIZE = 32
EPOCHS = 20
LEARNING_RATE = 0.001

In [23]:
# Direktori dataset
train_dir = "train"
test_dir = "test"

In [24]:
# Augmentasi Data
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

In [25]:
test_datagen = ImageDataGenerator(rescale=1.0/255)

In [26]:
# Generator Data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)


Found 6881 images belonging to 116 classes.


In [27]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

Found 1292 images belonging to 116 classes.


In [28]:
# Informasi jumlah kelas
num_classes = len(train_generator.class_indices)

In [29]:
# Model Transfer Learning
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Bekukan layer pada model dasar

In [30]:
# Tambahkan lapisan khusus
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)  # Dropout untuk mencegah overfitting
x = Dense(128, activation="relu")(x)
predictions = Dense(num_classes, activation="softmax")(x)

In [31]:
# Buat model
model = Model(inputs=base_model.input, outputs=predictions)

In [32]:
# Kompilasi model
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [33]:
# Training
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=test_generator
)

Epoch 1/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 237s 1s/step - accuracy: 0.1766 - loss: 3.7309 - val_accuracy: 0.4837 - val_loss: 1.8616
Epoch 2/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 224s 1s/step - accuracy: 0.5291 - loss: 1.6938 - val_accuracy: 0.6161 - val_loss: 1.3188
Epoch 3/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 227s 1s/step - accuracy: 0.6298 - loss: 1.2721 - val_accuracy: 0.6966 - val_loss: 1.0558
Epoch 4/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 228s 1s/step - accuracy: 0.6789 - loss: 1.0701 - val_accuracy: 0.7268 - val_loss: 0.9759
Epoch 5/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 229s 1s/step - accuracy: 0.7036 - loss: 0.9729 - val_accuracy: 0.7337 - val_loss: 0.8971
Epoch 6/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 229s 1s/step - accuracy: 0.7273 - loss: 0.8893 - val_accuracy: 0.7337 - val_loss: 0.9288
Epoch 7/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 227s 1s/step - accuracy: 0.7400 - loss: 0.8278 - val_accuracy: 0.7678 - val_loss: 0.7736
Epoch 8/20
216/216 ━━━━━━━━━━━━━━━━━━━━ 227s 1s/step - accuracy: 0.7619 - loss: 0.7632 - val_accu

In [34]:
model.save("modelr_11_20_epochs.h5")  # Menyimpan model ke file my_model.h5